# Finding Optimal Resolution

Quick an dirty script that tries to automatically find the optimal resolution for testing some image

## Find the Ideal Image Size for Training Data

`INPUT_IMAGE` - The image you want to run the analysis on

`training_avg_r` - The average ideal grain radius of your training data

`dim` - Starting resolution for the search

`negative` - Means image has black grains and white boundaries

In [20]:
from src.analysis.pixels_per_grain import pixels_per_grain

def find_ideal_size(img, target_r, start_dim=528, tolerance=2):
    dim = start_dim
    while True:
        _, avg_size, avg_r, _ = pixels_per_grain(img, (dim,dim))
        print(dim, avg_r)
        
        if np.isnan(avg_r):
            print("Average r is nan, something horrible has happened")
            return
        
        if np.abs(avg_r - target_r) < tolerance:
            return dim, avg_r, avg_size
        elif avg_r > target_r:
            dim -= 16
        elif avg_r < target_r:
            dim += 16

In [12]:
import os
from src.analysis.pixels_per_grain import pixels_per_grain
from skimage import io
import numpy as np
%load_ext autoreload
%autoreload 2

dir_path = '../data/test_combination/1HR/2789'
img_paths = ['trace.png','results/stackfirst_skel.png']
negative = [False, True]
crop = [True, True]
training_avg_r = 31
dim = 528
tolerance = 1

for ind, img_path in enumerate(img_paths):
    img = io.imread(os.path.join(dir_path, img_path))
    if len(img.shape) > 2:
        img = img[:,:,0]
    if negative[ind]:
        img = np.max(img) - img
    if crop[ind]:
        cf = img.shape[0]
        img_c = img[cf//4:-cf//4,cf//4:-cf//4]
        img = img_c
    
    print(os.path.join(dir_path, img_path))
    dim, avg_r, avg_size = find_ideal_size(img, training_avg_r, dim, tolerance)

    if crop[ind]:
        print(f"\nAt Res {2*dim}x{2*dim}")
        print(f"Average Radius: {avg_r}\nAverage Size: {avg_size}\n")
    else:
        print(f"\nAt Res {dim}x{dim}")
        print(f"Average Radius: {avg_r}\nAverage Size: {avg_size}\n")

../data/test_combination/1HR/2789/trace.png
528 60.409742726958235


NameError: name 'nan' is not defined

## Finding the ideal Grain Size for Multiple Images

Currently configured for a structure like
```
in_dir
+---fov1
¦   +---trace.png
¦   \---other_images.png
+---something else
+---fov2
¦   +---other_images.png
¦   \---trace.png
```
Where it'll only use `trace.png` in `fov1/` and `fov2/`

In [3]:
import os
import csv
from skimage import io
import numpy as np

in_dir = 'G:/My Drive/Lab/Grain-U-Net/data/test_Pt_Axon_3_denoised_transfer'
training_avg_r = 31

fovs = np.array([])
image_paths = np.array([])
for sub_dir in os.listdir(in_dir):
    if not os.path.isdir(os.path.join(in_dir, sub_dir)):
        continue
    for fname in os.listdir(os.path.join(in_dir, sub_dir)):
        if fname == 'trace.png':
            fovs = np.append(fovs, sub_dir)
            image_paths = np.append(image_paths, os.path.join(in_dir, sub_dir, fname))

dims = np.array([])
avg_rs = np.array([])
avg_sizes = np.array([])

for img_path in image_paths:
    print(img_path)
    img = io.imread(img_path)
    if len(img.shape) > 2:
        img = img[:,:,0]
        
    dim, avg_r, avg_size = find_ideal_size(img, training_avg_r, 1200)
    dims = np.append(dims, dim)
    avg_rs = np.append(avg_rs, avg_r)
    avg_sizes = np.append(avg_sizes, avg_size)
    
out = np.column_stack((fovs, dims, avg_rs, avg_sizes))
title = np.array(['FOV','Ideal Dimensions','Avg Grain r','Avg Grain Area'])
out = np.row_stack((title, out))
np.savetxt(os.path.join(in_dir,'ideal_res.csv'), out, delimiter=',', fmt='%s')

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', 'ideal_res.csv']


## Find the Average Ideal Grain Radius of Training Data

This find the `Average Pixels per Grain` and the `Average Ideal Grain Radius` for a set of training images

`INPUT_PATH` should be the location of the traced images in your training data.

In [11]:
import os
from src.analysis.grainpixels import pixels_per_grain
from skimage import io
import numpy as np
%load_ext autoreload
%autoreload 2

INPUT_PATH = '../data/train_nouveaux_256/label'

image_paths = np.array([])
for fname in os.listdir(INPUT_PATH):
    if fname.endswith('.png'):
        image_paths = np.append(image_paths, os.path.join(INPUT_PATH, fname))

data = np.array([0,0,0])

for ind, path in enumerate(image_paths):
    trace = io.imread(path)
    trace = (trace / 255)
    if len(trace.shape) > 2:
        trace = trace[:,:,0]

    (n_grains, avg, r_avg, r_std) = pixels_per_grain(trace, dims=(336,336), thresh = 0.6)
    if np.isnan(np.array([avg, r_avg, r_std])).any():
        continue
    if ind == 0:
        data = np.array([avg, r_avg, r_std])
    else:
        data = np.vstack((data, [avg, r_avg, r_std]))
    #print(f"Checking {path}")

print(f"\n\
Avg Pixels per Grain: {np.mean(data[:,0])}\n\
Avg Ideal Grain Radius: {np.mean(data[:,1])}\n\
Ideal Grain Radius STD: {np.mean(data[:,2])}")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

Avg Pixels per Grain: 3974.131416302186
Avg Ideal Grain Radius: 31.309408737097613
Ideal Grain Radius STD: 13.249901549722873
